In [13]:
import numpy as np
import matplotlib.pyplot as plt
import logging
from QLearningEGreedy import QLearningEGreedy as egreedy
import pandas as pd
import seaborn as sns

In [14]:
# RUN
#f_csma = "../../SOMAC-ML/data/_BKP/29092018/csma/round2/backlog_file.npy"
#f_tdma = "../../SOMAC-ML/data/_BKP/29092018/tdma/round2/backlog_file.npy"

# RUN3
#f_csma = "../../SOMAC-ML/data/10102018/round1/csma/backlog_file.npy"
#f_tdma = "../../SOMAC-ML/data/10102018/round1/tdma/backlog_file.npy"

# RUN4
f_csma = "../../SOMAC-ML/data/06102018/run4/5/csma/backlog_file.npy"
f_tdma = "../../SOMAC-ML/data/06102018/run4/5/tdma/backlog_file.npy"

d_csma = np.load(f_csma, encoding = "latin1").item()
d_tdma = np.load(f_tdma, encoding = "latin1").item()

n = np.min([len(d_csma), len(d_tdma)]) - 10

t_csma = np.array([d_csma[t]["metrics"][0, 1] for t in range(n)])
t_tdma = np.array([d_tdma[t]["metrics"][0, 1] for t in range(n)])

In [15]:
def calc_reward(curr, prev):
    # Positive reward
    if curr > prev and curr <= 1.2 * prev:
        reward = 0.
        
    elif curr > 1.2 * prev and curr <= 1.4 * prev:
        reward = 1.
        
    elif curr > 1.4 * prev and curr <= 1.6 * prev:
        reward = 2.
        
    elif curr > 1.6 * prev and curr <= 1.8 * prev:
        reward = 3.
        
    elif curr > 1.8 * prev and curr <= 2.0 * prev:
        reward = 4.
        
    elif curr > 2.0 * prev:
        reward = 5.

    # Negative reward
    elif prev > curr and prev <= 1.2 * curr:
        reward = 0.

    elif prev > 1.2 * curr and prev <= 1.4 * curr:
        reward = -1.
        
    elif prev > 1.4 * curr and prev <= 1.6 * curr:
        reward = -2.
        
    elif prev > 1.6 * curr and prev <= 1.8 * curr:
        reward = -3.
        
    elif prev > 1.8 * curr and prev <= 2.0 * curr:
        reward = -4.
        
    elif prev > 2.0 * curr:
        reward = -5.

    else:
        reward = 0.

    return reward

def test(learn_rate = 0.3, discount = 0.8, dic = {}):
    logging.basicConfig(filename="/Users/andre.gomes/Temp/out.log", level = logging.INFO)
    
    dic["learn_rate"].append(learn_rate)
    dic["discount"].append(discount)
    
    prot   = 1
    somac  = egreedy(prot, learn_rate = learn_rate, discount = discount)
    metric = {}
    t      = 0
    dt     = -1

    decision = prot
    backlog  = {}
    
    while t < n:
        metric[t] = t_csma[t] if prot == 0 else t_tdma[t]
        #print("Metrics = {}".format(metric[t]))
        #print("t = {}, dt = {}".format(t, dt))

        backlog[t] = prot

        if dt > 1:
            if dt == 2:
                reward = calc_reward(metric[t], metric[t-2])
            elif dt == 3:
                reward = calc_reward(metric[t], metric[t-3])
            else:
                reward = calc_reward(metric[t], metric[t-1]) * 0.01
                #reward = reward if reward == 0 else reward ** -dt

            somac.update_qtable(reward, dt)

            if dt == 2 and reward > 0:
                decision = somac.decision(prot, keep = True)
            elif dt == 2 and reward < 0.:
                decision = somac.decision(prot, force_switch = True)
            else:
                decision = somac.decision(prot, keep = False)

            if decision != prot:
                dt = 0
        else:
            _ = 0
            #print("MAC prot was switched recently. No switch now")


        t = t + 1
        dt = dt + 1

        #print("Protocol: {}".format(
        #    "CSMA" if prot == 0 else "TDMA"
        #))

        prot = decision

        #print("Decision: {}".format(
        #    "CSMA" if prot == 0 else "TDMA"
        #))

        #print("\n=============================\n")
        
    return backlog, metric
        
        
def calc_stats(backlog, metric, dic = {}):
    n_csma  = 0
    n_tdma  = 0
    n_somac = 0
    n_changes = 0
    
    per_threshold = 0.9
    dic["threshold"].append(per_threshold)
    
    for t in range(n):
        if t_csma[t] >= per_threshold * t_tdma[t]:
            n_csma = n_csma + 1

            if backlog[t] == 0:
                n_somac = n_somac + 1

        if t_tdma[t] >= per_threshold * t_csma[t]:
            n_tdma = n_tdma + 1

            if backlog[t] == 1:
                n_somac = n_somac + 1

    for t in range(1, n, 1):
        if backlog[t-1] != backlog[t]:
            n_changes = n_changes + 1
    
    #########################################################################

    norm = np.max([
        round(np.mean(t_csma), 2),
        round(np.mean(t_tdma), 2),
        round(np.mean([metric[t] for t in range(n)]), 2)
    ])
    
    dic["csma_count"].append(round(n_csma * 1. / n, 2))
    dic["tdma_count"].append(round(n_tdma * 1. / n, 2))
    dic["hit_rate"].append(round(n_somac * 1. / n, 2))
    dic["n_switches"].append(n_changes)
    dic["n_tot"].append(n)
    dic["csma_performance"].append(round(np.mean(t_csma), 2))
    dic["tdma_performance"].append(round(np.mean(t_tdma), 2))
    dic["somac_performance"].append(round(np.mean([metric[t] for t in range(n)]), 2))
    dic["csma_per_performance"].append(round(np.mean(t_csma) / norm, 2))
    dic["tdma_per_performance"].append(round(np.mean(t_tdma) / norm, 2))
    dic["somac_per_performance"].append(round(np.mean([metric[t] for t in range(n)]) / norm, 2))

In [16]:
fname = "./_tmp/run4.npy"

results = {
    "learn_rate":            [],
    "discount":              [],
    "threshold":             [],
    "csma_count":            [],
    "tdma_count":            [],
    "hit_rate":              [],
    "n_switches":            [],
    "n_tot":                 [],
    "csma_performance":      [],
    "tdma_performance":      [],
    "somac_performance":     [],
    "csma_per_performance":  [],
    "tdma_per_performance":  [],
    "somac_per_performance": []
}

for step in range(100):
    
    print("step: {}".format(step))
    
    learn_rate_list = [i/10. for i in range(11)]
    discount_list = [i/10. for i in range(11)]

    for learn_rate in learn_rate_list:
        for discount in discount_list:
            backlog, metric = test(learn_rate = learn_rate, discount = discount, dic = results)
            calc_stats(backlog, metric, dic = results)
            
np.save(fname, results)

step: 0
step: 1
step: 2
step: 3
step: 4
step: 5
step: 6
step: 7
step: 8
step: 9
step: 10
step: 11
step: 12
step: 13
step: 14
step: 15
step: 16
step: 17
step: 18
step: 19
step: 20
step: 21
step: 22
step: 23
step: 24
step: 25
step: 26
step: 27
step: 28
step: 29
step: 30
step: 31
step: 32
step: 33
step: 34
step: 35
step: 36
step: 37
step: 38
step: 39
step: 40
step: 41
step: 42
step: 43
step: 44
step: 45
step: 46
step: 47
step: 48
step: 49
step: 50
step: 51
step: 52
step: 53
step: 54
step: 55
step: 56
step: 57
step: 58
step: 59
step: 60
step: 61
step: 62
step: 63
step: 64
step: 65
step: 66
step: 67
step: 68
step: 69
step: 70
step: 71
step: 72
step: 73
step: 74
step: 75
step: 76
step: 77
step: 78
step: 79
step: 80
step: 81
step: 82
step: 83
step: 84
step: 85
step: 86
step: 87
step: 88
step: 89
step: 90
step: 91
step: 92
step: 93
step: 94
step: 95
step: 96
step: 97
step: 98
step: 99


In [17]:
df = pd.DataFrame.from_dict(results)

In [18]:
df

,learn_rate,discount,threshold,csma_count,tdma_count,hit_rate,n_switches,n_tot,csma_performance,tdma_performance,somac_performance,csma_per_performance,tdma_per_performance,somac_per_performance
0,0.0,0.0,0.9,0.57,0.51,0.70,24,88,9.59,10.31,11.45,0.84,0.90,1.00
1,0.0,0.1,0.9,0.57,0.51,0.70,24,88,9.59,10.31,11.45,0.84,0.90,1.00
2,0.0,0.2,0.9,0.57,0.51,0.70,24,88,9.59,10.31,11.45,0.84,0.90,1.00
3,0.0,0.3,0.9,0.57,0.51,0.70,24,88,9.59,10.31,11.45,0.84,0.90,1.00
4,0.0,0.4,0.9,0.57,0.51,0.70,24,88,9.59,10.31,11.45,0.84,0.90,1.00
5,0.0,0.5,0.9,0.57,0.51,0.70,24,88,9.59,10.31,11.45,0.84,0.90,1.00
6,0.0,0.6,0.9,0.57,0.51,0.70,24,88,9.59,10.31,11.45,0.84,0.90,1.00
7,0.0,0.7,0.9,0.57,0.51,0.70,24,88,9.59,10.31,11.45,0.84,0.90,1.00
8,0.0,0.8,0.9,0.57,0.51,0.70,24,88,9.59,10.31,11.45,0.84,0.90,1.00
9,0.0,0.9,0.9,0.57,0.51,0.70,24,88,9.59,10.31,11.45,0.84,0.90,1.00
